# Saving logits of original and adversarially attacked inputs

The "hack" below allows absolute path imports as if the notebook was a py-file run with `python -m abs.path.to.file.filename`

In [10]:
import sys
root_dir = !cd .. &&pwd
root_dir = root_dir[0] + "/"
sys.path.insert(0, root_dir)

In [11]:
print(root_dir)

/Users/gunder/Desktop/bachelor_project/


### Imports

In [12]:
import torch
import numpy as np
import matplotlib.pyplot as plt

from models.classifier32 import classifier32

from our_modules.tin_tools import get_avg_osr_auroc_across_splits
from our_modules.tin_tools import transform_range as tin_clip_range
from our_modules.tin_tools import save_grad_norms_across_splits
from our_modules.tin_tools import save_informed_attack

from our_modules.adv_tools import fp_osr_fgsm, fn_osr_fgsm, fp_osr_fgsm_sum_exp, fn_osr_fgsm_log_msp, fn_osr_fgsm_sum_exp
from our_modules.adv_tools import log_msp_loss
from our_modules.adv_tools import iterative_attack
from our_modules.adv_tools import norm_loss



### Setting the Device

In [13]:
if sys.platform == 'darwin':
    device = torch.device("mps")
else:
    gpu = torch.cuda.is_available()
    device = torch.device("cuda:0" if gpu else "cpu")

print("device:", device)

device: mps


## tiny-imagenet

In [14]:
path_to_pretrained_weights_folder = root_dir + "pretrained_weights/"
tin_val_root_dir = root_dir + "datasets/tiny-imagenet-200/val/images/"
tin_logits_dir = root_dir + "logits/tinyimagenet/"
tin_grad_norms_dir = root_dir + 'grad_norms/tinyimagenet/'

### Saving plain logits

In [15]:
#get_avg_osr_auroc_across_splits(path_to_pretrained_weights_folder, tin_val_root_dir, device, logdir=tin_logits_dir + "plain/")

### Savining Advesarial Attacks Logits

#### FGSM for different epsilon

##### FN

In [16]:
def fgsm_eps_experiment(eps_list, fgsm, logdir, number_of_splits=5, **fgsm_kwargs):
    for eps in eps_list:
        attack = (lambda yhat, y, model: fgsm(model, yhat, eps=eps, clip_range=tin_clip_range, **fgsm_kwargs))
        get_avg_osr_auroc_across_splits(path_to_pretrained_weights_folder, tin_val_root_dir, device, logdir + f"eps_{eps:.3}/", adv_attack=attack, number_of_splits=number_of_splits)


In [9]:
attack = (lambda x, y, model: iterative_attack(model, x, y, loss_func=lambda yhat, y: torch.max(yhat), torch_optim=torch.optim.Rprop, clip_range=tin_clip_range, return_step=False, max_iter=25, step_sizes=(1e-06, 5e-03)))
get_avg_osr_auroc_across_splits(path_to_pretrained_weights_folder, tin_val_root_dir, device, tin_logits_dir + 'fn/ia/test/', 20, shuffle=False, adv_attack=attack, number_of_splits=1)

  0%|          | 0/1 [00:00<?, ?it/s]

tensor(3.1233e-05, device='mps:0', grad_fn=<MaxBackward1>)
tensor(0., device='mps:0', grad_fn=<MaxBackward1>)
tensor(0.0086, device='mps:0', grad_fn=<MaxBackward1>)
tensor(0.0086, device='mps:0', grad_fn=<MaxBackward1>)
tensor(0.0194, device='mps:0', grad_fn=<MaxBackward1>)
tensor(0., device='mps:0', grad_fn=<MaxBackward1>)
tensor(0.0194, device='mps:0', grad_fn=<MaxBackward1>)
tensor(0.0194, device='mps:0', grad_fn=<MaxBackward1>)
tensor(0.0194, device='mps:0', grad_fn=<MaxBackward1>)
tensor(0.0194, device='mps:0', grad_fn=<MaxBackward1>)
tensor(0.0194, device='mps:0', grad_fn=<MaxBackward1>)
tensor(0.0194, device='mps:0', grad_fn=<MaxBackward1>)
tensor(0., device='mps:0', grad_fn=<MaxBackward1>)
tensor(0.0086, device='mps:0', grad_fn=<MaxBackward1>)
tensor(0.0194, device='mps:0', grad_fn=<MaxBackward1>)
tensor(0.0194, device='mps:0', grad_fn=<MaxBackward1>)
tensor(0.0194, device='mps:0', grad_fn=<MaxBackward1>)
tensor(0.0194, device='mps:0', grad_fn=<MaxBackward1>)
tensor(0., device=

  0%|          | 0/1 [00:26<?, ?it/s]

tensor(0.0194, device='mps:0', grad_fn=<MaxBackward1>)


KeyboardInterrupt: 

In [ ]:
eps = np.array([1.2, 1.4, 1.6, 1.8, 2.0, 2.25, 2.50, 2.75, 3.0, 4.0])

In [ ]:
eps

array([1.2 , 1.4 , 1.6 , 1.8 , 2.  , 2.25, 2.5 , 2.75, 3.  , 4.  ])

In [2]:
fig, ax = plt.subplots(1,1)
ax.scatter(eps, np.zeros_like(eps), marker ='|')
plt.show()

NameError: name 'plt' is not defined

In [32]:
#fgsm_eps_experiment(eps, fn_osr_fgsm, tin_logits_dir + "fn/fgsm/", 1)

#### FP

2-norm

In [33]:
# fgsm_eps_experiment(eps_list=eps, fgsm=fp_osr_fgsm, logdir=tin_logits_dir + "fp/fgsm/two_norm/", number_of_splits=1, norm_ord=None)
# fgsm_eps_experiment(eps_list=eps, fgsm=fp_osr_fgsm, logdir=tin_logits_dir + "fp/fgsm/inf_norm/", number_of_splits=1, norm_ord=torch.inf)
# fgsm_eps_experiment(eps_list=eps, fgsm=fp_osr_fgsm_sum_exp, logdir=tin_logits_dir + "fp/fgsm/sum_exp/", number_of_splits=1)
#fgsm_eps_experiment(eps_list=eps, fgsm=fn_osr_fgsm, logdir=tin_logits_dir + "fn/fgsm/inf_norm/", number_of_splits=1, norm_ord=torch.inf)
#fgsm_eps_experiment(eps_list=eps, fgsm=fn_osr_fgsm_log_msp, logdir=tin_logits_dir + "fn/fgsm/log_msp/", number_of_splits=1)
#fgsm_eps_experiment(eps_list=eps, fgsm=fn_osr_fgsm_sum_exp, logdir=tin_logits_dir + "fn/fgsm/sum_exp/", number_of_splits=1)

100%|██████████| 1/1 [01:36<00:00, 96.41s/it]


### Saving Grad Norms 
Testing Odin paper

In [8]:
#save_grad_norms_across_splits(path_to_pretrained_weights_folder, tin_val_root_dir, tin_grad_norms_dir + 'log_msp/', log_msp_loss, device, number_of_splits=5)

### Informed Attack

In [9]:
save_informed_attack(tin_logits_dir + 'informed/fgsm/exp1/',
                     tin_logits_dir + 'fn/fgsm/inf_norm/eps_0.139/', 
                     tin_logits_dir + 'fp/fgsm/two_norm/eps_0.0622/', 0)
save_informed_attack(tin_logits_dir + 'informed/fgsm/exp2/', 
                     tin_logits_dir + 'plain/', 
                     tin_logits_dir + 'fp/fgsm/two_norm/eps_0.0622/', 0)
save_informed_attack(tin_logits_dir + 'informed/fgsm/exp3/',
                     tin_logits_dir + 'fn/fgsm/inf_norm/eps_0.139/', 
                     tin_logits_dir + 'plain/', 0)